# Google API GEMINI
 * config_API.json  with 
```text
{
  "GEMINI_API_KEY": "Your Key"
} 
```

In [ ]:
# functional
import os
import gradio as gr
import google.generativeai as genai
import traceback
import json # For loading config_API.json

# --- Your Proxy Setup (for external calls like Google Gemini) ---
PROXY_URL = os.environ.get("HTTP_PROXY_URL", "http://localhost:4321")
if PROXY_URL:
    os.environ['http_proxy'] = PROXY_URL
    os.environ['https_proxy'] = PROXY_URL
    print(f"Using HTTP_PROXY: {os.environ.get('http_proxy')}")
    print(f"Using HTTPS_PROXY: {os.environ.get('https_proxy')}")
else:
    print("No HTTP_PROXY_URL environment variable set, not using proxy for HTTP/HTTPS.")

# --- CRITICAL: Add NO_PROXY for local Gradio communication ---
current_no_proxy = os.environ.get('NO_PROXY', '')
additional_no_proxy_hosts = ['localhost', '127.0.0.1', '0.0.0.0']
new_no_proxy_parts = [host for host in current_no_proxy.split(',') if host.strip()]
for host in additional_no_proxy_hosts:
    if host not in new_no_proxy_parts:
        new_no_proxy_parts.append(host)
os.environ['NO_PROXY'] = ','.join(new_no_proxy_parts)
print(f"Using NO_PROXY: {os.environ.get('NO_PROXY')}")
# --- END Proxy Setup ---

# --- Global Variables ---
model = None
current_model_name = None
genai_configured_successfully = False # Flag to check if genai.configure() was successful
system_instruction = "You are a helpful and friendly AI assistant. Be concise but informative."

# --- Available Models ---
AVAILABLE_MODELS = {
    "Gemini 1.5 Flash (Fast, Cost-Effective)": "gemini-1.5-flash-latest",
    "Gemini 1.5 Pro (Most Capable)": "gemini-1.5-pro-latest",
    "Gemini 1.0 Pro (General Purpose)": "gemini-1.0-pro",
}
DEFAULT_MODEL_DISPLAY_NAME = "Gemini 1.5 Flash (Fast, Cost-Effective)"
initial_model_technical_name = AVAILABLE_MODELS.get(DEFAULT_MODEL_DISPLAY_NAME, "gemini-1.5-flash-latest")


# --- Configure Gemini API Key ---
def configure_gemini_api():
    global genai_configured_successfully
    api_key_to_use = None
    source = None

    # 1. Try environment variable
    env_api_key = os.environ.get("GEMINI_API_KEY")
    if env_api_key:
        api_key_to_use = env_api_key
        source = "environment variable"
    # 2. If not found, try config_API.json
    elif os.path.exists('config_API.json'):
        try:
            with open('config_API.json', 'r') as f:
                config = json.load(f)
            file_api_key = config.get("GEMINI_API_KEY")
            if file_api_key:
                api_key_to_use = file_api_key
                source = "config_API.json"
            else:
                print("ERROR: GEMINI_API_KEY not found in config_API.json.")
        except (FileNotFoundError, json.JSONDecodeError) as e: # Should not happen due to os.path.exists but good practice
            print(f"Error loading or parsing config_API.json: {e}")
        except Exception as e:
            print(f"Unexpected error reading config_API.json: {e}")
            traceback.print_exc()
    else:
        print("ERROR: GEMINI_API_KEY environment variable not set and config_API.json not found.")

    if api_key_to_use:
        try:
            genai.configure(api_key=api_key_to_use)
            genai_configured_successfully = True
            print(f"Google Gemini API configured successfully using {source}.")
        except Exception as e:
            print(f"ERROR: Failed to configure Gemini API using key from {source}: {e}")
            traceback.print_exc()
            genai_configured_successfully = False
    else:
        print("ERROR: No API key found to configure Google Gemini.")
        genai_configured_successfully = False

configure_gemini_api() # Attempt to configure on script start


# --- Initialize/Switch Gemini Model ---
def initialize_gemini_model(model_name_to_load):
    global model, current_model_name
    if not genai_configured_successfully:
        msg = "Error: Gemini API not configured. Check API Key source (env var or config_API.json)."
        print(msg)
        model = None
        current_model_name = None
        return msg

    print(f"Attempting to initialize model: {model_name_to_load}")
    try:
        model = genai.GenerativeModel(
            model_name=model_name_to_load,
            system_instruction=system_instruction
        )
        current_model_name = model_name_to_load
        success_msg = f"Successfully switched to model: {current_model_name}"
        print(success_msg)
        return success_msg
    except Exception as e:
        error_msg = f"Error initializing model '{model_name_to_load}': {e}"
        print(error_msg)
        traceback.print_exc()
        model = None # Ensure model is None if initialization fails
        current_model_name = None
        return error_msg

# --- Initialize with a default model on startup (if API was configured) ---
if genai_configured_successfully:
    initialize_gemini_model(initial_model_technical_name)
else:
    print("Skipping initial model load as Gemini API is not configured.")


def gemini_chat_responder(user_message, chat_history_gradio):
    global model
    if not genai_configured_successfully:
         return ("Error: Google Gemini API is not configured. "
                "Please ensure GEMINI_API_KEY is set in your environment or in config_API.json.")
    if model is None:
        return ("Error: Google Gemini model is not initialized or failed to switch. "
                "Please check API key, select a model from the dropdown, and see server logs.")

    if not user_message.strip():
        return "Please type a message."

    gemini_history = []
    for user_msg, ai_msg in chat_history_gradio:
        if user_msg:
            gemini_history.append({'role': 'user', 'parts': [{'text': user_msg}]})
        if ai_msg:
            gemini_history.append({'role': 'model', 'parts': [{'text': ai_msg}]})

    gemini_history.append({'role': 'user', 'parts': [{'text': user_message}]})

    try:
        print(f"Sending to Gemini ({model.model_name}): {gemini_history[-1]}")
        response = model.generate_content(
            gemini_history,
            generation_config=genai.types.GenerationConfig(
                max_output_tokens=2048,
                temperature=0.7,
                top_p=0.9,
                top_k=40
            ),
        )
        if not response.candidates:
            block_reason = response.prompt_feedback.block_reason if response.prompt_feedback else "Unknown"
            block_message = f"The response was blocked. Reason: {block_reason}."
            if response.prompt_feedback and response.prompt_feedback.safety_ratings:
                block_message += f" Safety ratings: {response.prompt_feedback.safety_ratings}"
            print(f"Gemini Error: {block_message}")
            return f"Error: {block_message} Please try rephrasing your message."

        assistant_message = response.candidates[0].content.parts[0].text
        print(f"Received from Gemini: {assistant_message[:100]}...")
        return assistant_message
    except genai.types.BlockedPromptError as e:
        error_msg = f"Gemini Error: Your prompt was blocked. {e}"
        print(error_msg)
        return error_msg
    except google.auth.exceptions.RefreshError as e:
        error_msg = f"Gemini Authentication Error: {e}. Please check your API key and ensure it's valid and has permissions."
        print(error_msg)
        return error_msg
    except google.api_core.exceptions.ResourceExhausted as e:
        error_msg = f"Gemini API Error: Rate limit exceeded or quota exhausted. {e}"
        print(error_msg)
        return error_msg
    except google.api_core.exceptions.GoogleAPIError as e:
        error_msg = f"Gemini API Error: {type(e).__name__} - {e}"
        print(error_msg)
        traceback.print_exc()
        return error_msg
    except Exception as e:
        error_msg = f"An unexpected error occurred with Google Gemini: {type(e).__name__} - {str(e)}"
        print(error_msg)
        traceback.print_exc()
        return error_msg

# --- Gradio UI with Blocks ---
with gr.Blocks(theme="soft", title="Gemini Chat Dashboard") as demo:
    gr.Markdown("# Google Gemini Chat Dashboard")
    gr.Markdown("Chat with an AI assistant powered by Google Gemini. Select your desired model below.")

    with gr.Row():
        model_dropdown = gr.Dropdown(
            choices=list(AVAILABLE_MODELS.keys()),
            value=DEFAULT_MODEL_DISPLAY_NAME,
            label="Select LLM Model",
            interactive=True # Will be set to False if API not configured
        )
        model_status_text = gr.Textbox(
            label="Model Status",
            value=(f"Current model: {current_model_name}" if current_model_name else "API Not Configured or Model Not Initialized"),
            interactive=False,
            max_lines=1
        )

    # The ChatInterface itself, embedded in Blocks
    chat_interface = gr.ChatInterface(
        fn=gemini_chat_responder,
        chatbot=gr.Chatbot(height=500, label="Conversation", show_copy_button=True, avatar_images=("user.png", "bot.png")),
        textbox=gr.Textbox(placeholder="Type your message here and press Enter...", lines=3, label="Your Message"),
        examples=[
            "Hello, how are you today?",
            "What's the capital of France?",
            "Write a short poem about a curious cat."
        ],
        retry_btn="Retry",
        undo_btn="Undo Last Turn",
        clear_btn="Clear Conversation",
    )

    # --- Event Handler for Dropdown Change ---
    def handle_model_change(selected_display_name):
        if not genai_configured_successfully:
            return "Cannot change model: Gemini API not configured."
        technical_name = AVAILABLE_MODELS.get(selected_display_name)
        if technical_name:
            feedback_message = initialize_gemini_model(technical_name)
            return feedback_message
        return "Error: Selected model name not found in configuration."

    model_dropdown.change(
        fn=handle_model_change,
        inputs=[model_dropdown],
        outputs=[model_status_text]
    )

    # If API is not configured, disable the dropdown and update status
    if not genai_configured_successfully:
        model_dropdown.interactive = False # This doesn't work directly after Blocks creation
                                            # Need to update properties dynamically or handle it in the UI logic
        gr.Markdown("### <p style='color:red;'>API Key Not Configured. Chat functionality will be disabled.</p>")
        # A more Gradio-idiomatic way to update interactivity is through an event,
        # but for initial setup, this print and the checks in responders are key.
        # You could also update the model_status_text and model_dropdown.interactive
        # using a dummy gr.Button.click() event that fires on load, but that's more complex.
        # The current checks in `gemini_chat_responder` and `handle_model_change`
        # will prevent operations if `genai_configured_successfully` is False.
        # The `model_status_text.value` is also set appropriately at the start.


if __name__ == "__main__":
    print("Launching Gradio app...")
    if not genai_configured_successfully:
        print("\nWARNING: Google Gemini API key not found or configuration failed.")
        print("The Gradio interface will launch, but chat functionality will be impaired until the API is configured and a model is loaded.\n")
    elif model is None: # API configured, but initial model load failed
        print("\nWARNING: Google Gemini API configured, but initial model failed to load.")
        print("Chat functionality may be impaired. Check logs and try selecting a model from the dropdown.\n")


    try:
        demo.launch(server_name="0.0.0.0", debug=True, share=False)
    except Exception as e:
        print(f"Failed to launch Gradio interface: {e}")
        traceback.print_exc()

Using HTTP_PROXY: http://localhost:4321
Using HTTPS_PROXY: http://localhost:4321
Using NO_PROXY: localhost,127.0.0.1,0.0.0.0
Google Gemini API configured successfully using config_API.json.
Attempting to initialize model: gemini-1.5-flash-latest
Successfully switched to model: gemini-1.5-flash-latest
Launching Gradio app...
Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
